# What it is

A script that takes an excel xlsx file containing the url to check and canoncial url pairings that a user wants to check. The tool outputs whether the canonical has been set at all, incorrectly, or correctly.

In [1]:
# User input data
# Currently the fields are set to be 

CANONICALS_WORKBOOK = 'Example.xlsx'
CANONICALS_WORKBOOK_SHEET = 'CanonicalsSheet'


In [2]:
# Imports and constants

import urllib.request as request
import pandas as pd
import re
import xlrd
from xlutils.copy import copy
from datetime import datetime
import requests
import urllib.request
import urllib
import pandas as pd
from bs4 import BeautifulSoup

import matplotlib
CANONICALS_INPUT_FOLDER = 'Inputs/'
CANONICALS_OUTPUT_FOLDER = 'Results/'

CANONICALS_INPUT_WORKBOOK_PATH = CANONICALS_INPUT_FOLDER + CANONICALS_WORKBOOK

to_check = xlrd.open_workbook(CANONICALS_INPUT_WORKBOOK_PATH)
to_check_sheet = to_check.sheet_by_name(CANONICALS_WORKBOOK_SHEET)

check_wb = copy(to_check) 
check_sheet = check_wb.get_sheet(CANONICALS_WORKBOOK_SHEET)

NO_CANONICAL_SET_STRING = "No canonical set"

In [65]:
# Methods to parse data in file

def return_canonical(url):
    f = urllib.request.urlopen(url)
    soup = BeautifulSoup(f.read(), 'html.parser')

    list_of_canonical = []
    #parse this by only selecting the image with the web url in the name
    for link in soup.findAll('link'):
        if link.get("rel")[0] == "canonical":
            list_of_canonical.append(link.get('href'))

    if len(list_of_canonical) == 1:
        return(list_of_canonical[0])
    elif len(list_of_canonical) == 0:
        return NO_CANONICAL_SET_STRING

def add_https_if_none(url):
    if url.startswith("www"):
        return "https://" + url
    else: return url

In [61]:
# Testing
def test_return_canonical(url):
    result = return_canonical(url)
    if result == "http://www.masterlock.com/business-use/product/A1266NBLK":
        return True
    else:
        print("Error when parsing")
        return result

test_return_canonical("https://www.masterlock.com/business-use/product/A1266NBLK")   

added


True

In [68]:
# Checking the canonicals

# These are what the headers of the outputted xlsx will be, along with the output printed after running this cell.
cols = ["result", "status code", "canonical status code", "url", "expected canonical", "actual canonical"]

# This will be the ouputted table that will hold all of the results.
list_of_results = pd.DataFrame(columns=cols)

# For every row in the input data, check to see that the canonical 1) exists 2) is what was desired
for i in range(1, len(check_sheet.rows)):
    
    url_containing_canonical = add_https_if_none(to_check_sheet.cell(i, 0).value.strip())
    expected_canonical = add_https_if_none(to_check_sheet.cell(i, 1).value.strip())
    
    req = requests.get(url_containing_canonical)
    
    # This status code is reflective of the last code outputted and will not reflect redirects
    status_code = req.status_code
    canonical_status_code = requests.get(expected_canonical).status_code
    if status_code in [200, 301]:
        actual_canonical = return_canonical(url_containing_canonical)

        if actual_canonical == NO_CANONICAL_SET_STRING:
            result = actual_canonical
        elif actual_canonical == expected_canonical:
            if canonical_status_code == 200:
                result = "OK"
            elif canonical_status_code == 301:
                result = "OK, but warning, canonical is a redirect."
            else:
                result = "Canonical is bad link, but is equal to expected. Consider changing canonical."
        else:
            result = 'Expected and actual canonicals do not match'
    else: 
        result = "Error when accessing url containing canonical. See status code."

    
    # Append the result to a dataframe for output later
    list_of_results.loc[i] = [result, status_code, canonical_status_code, 
                              url_containing_canonical, expected_canonical, actual_canonical]

print(list_of_results)

                                         result status code  \
1                              No canonical set         200   
2                              No canonical set         200   
3   Expected and actual canonicals do not match         200   
4   Expected and actual canonicals do not match         200   
5   Expected and actual canonicals do not match         200   
6   Expected and actual canonicals do not match         200   
7   Expected and actual canonicals do not match         200   
8   Expected and actual canonicals do not match         200   
9   Expected and actual canonicals do not match         200   
10  Expected and actual canonicals do not match         200   
11  Expected and actual canonicals do not match         200   
12  Expected and actual canonicals do not match         200   
13  Expected and actual canonicals do not match         200   
14  Expected and actual canonicals do not match         200   
15  Expected and actual canonicals do not match        

In [69]:
# Run to output the dataframe as an xlsx file in the 'Results' folder

OUTPUT_FILE = CANONICALS_OUTPUT_FOLDER + 'canonical-results_'+ datetime.now().strftime("%Y-%m-%d_%H-%M") + '.xlsx'

writer = pd.ExcelWriter(OUTPUT_FILE, engine='xlsxwriter',)
list_of_results.to_excel(writer, sheet_name='Canonicals', index=False)
writer.save()